<a href="https://colab.research.google.com/github/DataParadox/iTrustBD/blob/main/Codes/iTrustBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Project from GitHub and Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/'

/content/drive/My Drive/PG/CSE6413 Network Science/Project Works


In [ ]:
!git clone 'https://github.com/DataParadox/iTrustBD.git'

In [3]:
%cd iTrustBD

/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/iTrustBD


In [5]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/DataParadox/iTrustBD
   72b4f91..2f6da27  main       -> origin/main
Updating 72b4f91..2f6da27
Fast-forward
 Codes/iTrustBD.ipynb | 150 ++++++++++++++++++++++++++++-----------------------
 1 file changed, 84 insertions(+), 66 deletions(-)


# Set Datasets

In [64]:
def setDatasets(dataset):           # dataset -> Name of the dataset, i.e. 'soc-sign-bitcoinalpha-dataset' 
  import pandas as pd
  
  nodeTable = pd.read_csv('./BTC-Alpha/'+dataset+'-nodes-table.csv')
  edgeTable = pd.read_csv('./BTC-Alpha/'+dataset+'-edge-table.csv')
  nodeTable['behavior'] = nodeTable['Class']

  edgeTableTargetSource = edgeTable[['Target', 'Source']].astype(str)
  neighborTableIn = edgeTableTargetSource.groupby(['Target'], as_index=False).agg(lambda col: ', '.join(col))
  neighborTableIn.columns = neighborTableIn.columns.str.replace('Source', 'Neighbors')

  neighborTableOut = edgeTableTargetSource.groupby(['Source'], as_index=False).agg(lambda col: ', '.join(col))
  neighborTableOut.columns = neighborTableOut.columns.str.replace('Target', 'Neighbors')

  return nodeTable, edgeTable, neighborTableIn, neighborTableOut


dataset = 'soc-sign-bitcoinalpha-dataset'
nodeTable, edgeTable, neighborTableIn, neighborTableOut = setDatasets(dataset)

# Generate Random Probability

In [65]:
def randProbability(min, max):
  # generate random floating point values
  import random
  return min + (random.random() * (max - min))

# Behavior Spread Based on Mutation Factor

In [66]:
def behaviorSpreadMutate(nodeTable):
  import random
  meu_m = randProbability(0, 0.01)                                     # Random mutation factor generation in the range of (0, 0.01)
  for trader in nodeTable.index:
    if nodeTable.iloc[trader].Class == 0:
      if randProbability(0, 1) >= (1-meu_m):                           # If cross the threshold of meu_m
        nodeTable.iloc[trader].behavior = random.choice([-1, 0, 1])   # Change behavior randomly

# Behavior Spread Based on Neighbors

In [67]:
def behaviorSpreadNeighbors(nodeTable, neighborTableIn):
  import random
  gamma = randProbability(0, 0.05)       # Random transmission rate generation in the range of (0, 0.05)
  for trader in nodeTable.index:
    if nodeTable.iloc[trader].Class == 0:
      if randProbability(0, 1) >= (1-gamma):
        try:
          random_neighbor = random.choice(list(map(int, neighborTableIn.loc[neighborTableIn['Target'].astype(int) == nodeTable.iloc[trader].Id.astype(int)]['Neighbors'].item().split(","))))
          #print(random_neighbor, 'A', list(map(int, neighborTableIn.loc[neighborTableIn['Target'].astype(int) == nodeTable.iloc[trader].Id.astype(int)]['Neighbors'].item().split(","))))
          nodeTable.iloc[trader].behavior = int(nodeTable.loc[nodeTable['Id'].astype(int) == int(random_neighbor)].behavior.to_string(index=False))
        except:
          pass

# Setting of Parameters

In [68]:
def setParameters():
  beta_T = randProbability(0, 1)
  beta_S = beta_T

  meu_T = randProbability(0, beta_T)
  meu_S = meu_T

  return beta_T, beta_S, meu_T, meu_S

# Calculating Propotion of Neighbors

In [69]:
def neighborProportion(nodeId, nodeTable, neighborTableIn, neighborTableOut):
  setNeighbor = set(list(map(int, neighborTableIn.loc[neighborTableIn['Target'].astype(int) == int(nodeId)]['Neighbors'].item().split(","))))
  setNeighbor = set(list(map(int, neighborTableOut.loc[neighborTableOut['Source'].astype(int) == int(nodeId)]['Neighbors'].item().split(",")))).union(setNeighbor)

  countTrusty = 0
  countSuspecious = 0
  countNormal = 0

  for neighbor in setNeighbor:
    if int(nodeTable.loc[nodeTable['Id'].astype(int) == int(neighbor)].behavior.to_string(index=False)) == 0:
      countNormal = countNormal + 1
    elif int(nodeTable.loc[nodeTable['Id'].astype(int) == int(neighbor)].behavior.to_string(index=False)) == 1:
      countTrusty = countTrusty + 1
    else:
      countSuspecious = countSuspecious + 1
  
  return countTrusty/(countTrusty+countNormal+countSuspecious), countNormal/(countTrusty+countNormal+countSuspecious), countSuspecious/(countTrusty+countNormal+countSuspecious)
  #return sigma_T, sigma_N, sigma_S

# Susceptible-Infected-Susceptible (SIS) Model

In [ ]:
def modelSIS():
  print('Codes goes here')              # Codes goes here
modelSIS()

Codes goes here


# Voter Model

In [ ]:
def modelVoter():
  print('Codes goes here')              # Codes goes here
modelVoter()

Codes goes here


# Bilingual Model

In [ ]:
def modelBilingual():
  print('Codes goes here')              # Codes goes here
modelBilingual()

Codes goes here


# Behavior Spread

In [ ]:
def behaviorSpread():
  epoch = 0
  total_epoch = 10000
  # Set all nodes ignorant
  # Set one random node aware
  while epoch < total_epoch:
    epoch = epoch + 1
    behaviorSpreadNeighbors()
    behaviorSpreadMutate()
    #for i in neutral_traders:
      # Select a random neighbor j with probability proportional to in_degree.
      #if i neutral & j not neutral:
        # i can assume the state of j under the rules of the model
      #elif i not neutral:
          # set i neutral under the rules of the model